In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from nb_005 import *

# Carvana

## Setup

(See final section of notebook for one-time data processing steps.)

In [ ]:
PATH = Path('data/carvana')
PATH_PNG = PATH/'train_masks_png'
PATH_X_FULL = PATH/'train'
PATH_X_128 = PATH/'train-128'
PATH_Y_FULL = PATH_PNG
PATH_Y_128 = PATH/'train_masks-128'

# start with the 128x128 images
PATH_X = PATH_X_128
PATH_Y = PATH_Y_128

In [ ]:
img_f = next(PATH_X.iterdir())
x = open_image(img_f)
show_image(x)

In [ ]:
#export
def pil2tensor(image, as_mask=False):
    arr = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))
    arr = arr.view(image.size[1], image.size[0], -1)
    arr = arr.permute(2,0,1).float()
    return arr if as_mask else arr.div_(255)

def open_image(fn, as_mask=False):
    x = PIL.Image.open(fn)
    if not as_mask: x = x.convert('RGB')
    return pil2tensor(x, as_mask=as_mask)

def image2np(image):
    res = image.cpu().permute(1,2,0).numpy()
    return res[...,0] if res.shape[2]==1 else res

def show_image(img, ax=None, figsize=(3,3), hide_axis=True, cmap='binary', alpha=None):
    if ax is None: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(image2np(img), cmap=cmap, alpha=alpha)
    if hide_axis: ax.axis('off')
    return ax
        
def show_xy_image(xim, yim, ax=None, figsize=(3,3), alpha=0.5, hide_axis=True, cmap='RdGy_r'):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax1 = show_image(xim, ax=ax, hide_axis=hide_axis, cmap=cmap)
    show_image(yim, ax=ax1, alpha=alpha, hide_axis=hide_axis, cmap=cmap)
    if hide_axis: ax.axis('off')
        
def show_xy_images(x,y,rows,figsize=(9,9)):
    fig, axs = plt.subplots(rows,rows,figsize=figsize)
    for i, ax in enumerate(axs.flatten()):
        show_xy_image(x[i], y[i], ax)
    plt.tight_layout()

    
class ImageMask(Image):
    # no lighting changes to masks
    # ??? should it actually be no logit changes?
    def lighting(self, func, *args, **kwargs):
        return self
    
    def refresh(self):
        # always use mode nearest for mask sampling
        force_nearest = {'mode':'nearest'}
        self.sample_kwargs = {**self.sample_kwargs, **force_nearest}
        super().refresh()
        return self
    
    
import nb_002b
nb_002b.pil2tensor = pil2tensor
nb_002b.open_image = open_image
nb_002b.image2np = image2np
nb_002b.show_image = show_image

In [ ]:
def get_y_fn(x_fn): return PATH_Y/f'{x_fn.name[:-4]}_mask.png'

img_y_f = get_y_fn(img_f)
y = open_image(img_y_f, as_mask=True)
show_image(y)

In [ ]:
show_xy_image(x,y)

## xy transforms

- data types: regr, class, seg, bbox, polygon, generative (s/res, color), custom

In [ ]:
#export
def apply_tfms(tfms, x, do_resolve=True, xtra=None, size=None,
               mult=32, do_crop=True, padding_mode='reflect',**kwargs):
    if not tfms: return x
    if not xtra: xtra={}
    tfms = sorted(listify(tfms), key=lambda o: o.tfm.order)
    if do_resolve: resolve_tfms(tfms)
    x.set_sample(padding_mode=padding_mode, **kwargs)
    if size:
        crop_target = get_crop_target(size, mult=mult)
        target = get_resize_target(x, crop_target, do_crop=do_crop)
        x.resize(target)

    size_tfms = [o for o in tfms if isinstance(o.tfm,TfmCrop)]
    for tfm in tfms:
        if tfm.tfm in xtra: x = tfm(x, **xtra[tfm.tfm])
        elif tfm in size_tfms: x = tfm(x, size=size, padding_mode=padding_mode)
        else: x = tfm(x)
    return x.px

import nb_003
nb_003.apply_tfms = apply_tfms

class DatasetTfm(Dataset):
    def __init__(self, ds:Dataset,tfms:Collection[Callable]=None,
                 x_class=Image, y_class=None, tfm_y:bool=False, **kwargs):
        self.ds,self.tfms,self.x_class,self.y_class,self.tfm_y,self.x_kwargs = ds,tfms,x_class,y_class,tfm_y,kwargs
        self.y_kwargs = {**self.x_kwargs, **{'do_resolve':False}} # don't reset random vars
        
    def __len__(self): return len(self.ds)
    
    def __getitem__(self,idx):
        x,y = self.ds[idx]
        
        if self.tfms is not None: 
            if self.x_class: x = self.x_class(x)
            x = apply_tfms(self.tfms, x, **self.x_kwargs)
            if self.tfm_y:
                if self.y_class: y = self.y_class(y)
                y = apply_tfms(self.tfms, y, **self.y_kwargs)
        return x,y
    
import nb_002b
nb_002b.DatasetTfm = DatasetTfm
    
def transform_datasets(train_ds, valid_ds, tfms, tfm_y=False, x_class=Image, y_class=None, size=None):
    ttds = DatasetTfm(train_ds, tfms[0], tfm_y=tfm_y, x_class=x_class, y_class=y_class, size=size)
    vtds = DatasetTfm(valid_ds, tfms[1], tfm_y=tfm_y, x_class=x_class, y_class=y_class, size=size)
    return ttds, vtds

import nb_005
nb_005.transform_datasets = transform_datasets

## Dataset

In [ ]:
#export
@dataclass
class MatchedFilesDataset(Dataset):
    x_fns:List[Path]; y_fns:List[Path]
    def __post_init__(self): assert len(self.x_fns)==len(self.y_fns)
    def __repr__(self): return f'{type(self).__name__} of len {len(self.x_fns)}'
    def __len__(self): return len(self.x_fns)
    def __getitem__(self, i): 
        return open_image(self.x_fns[i]), open_image(self.y_fns[i],as_mask=True)
    
def split_by_idxs(seq, idxs):
    '''A generator that returns sequence pieces, seperated by indexes specified in idxs. '''
    last = 0
    for idx in idxs:
        if not (-len(seq) <= idx < len(seq)):
            raise KeyError(f'Idx {idx} is out-of-bounds')
        yield seq[last:idx]
        last = idx
    yield seq[last:]

def split_by_idx(idxs, *a):
    """
    Split each array passed as *a, to a pair of arrays like this (elements selected by idxs,  the remaining elements)
    This can be used to split multiple arrays containing training data to validation and training set.
    :param idxs [int]: list of indexes selected
    :param a list: list of np.array, each array should have same amount of elements in the first dimension
    :return: list of tuples, each containing a split of corresponding array from *a.
            First element of each tuple is an array composed from elements selected by idxs,
            second element is an array of remaining elements.
    """
    mask = np.zeros(len(a[0]),dtype=bool)
    mask[np.array(idxs)] = True
    return [(o[mask],o[~mask]) for o in a]

In [ ]:
x_fns = [o for o in PATH_X.iterdir() if o.is_file()]
y_fns = [get_y_fn(o) for o in x_fns]
val_idxs = list(range(1008))
((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, np.array(x_fns), np.array(y_fns))
train_ds = MatchedFilesDataset(trn_x, trn_y)
valid_ds = MatchedFilesDataset(val_x, val_y)
train_ds, valid_ds

In [ ]:
x,y = next(iter(train_ds))
x.shape, y.shape

In [ ]:
size=128
tfms = get_transforms(do_flip=True, max_rotate=20, max_zoom=2., max_lighting=0.7, max_warp=0.3,p_affine=0.75)
train_tds, valid_tds = transform_datasets(train_ds, valid_ds, tfms, tfm_y=True, y_class=ImageMask, size=size)

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,6))
for i in range(4):
    imgx,imgy = train_tds[i]
    show_xy_image(imgx, imgy, axes[i])

In [ ]:
#export
def normalize(x, mean,std):   return (x-mean[...,None,None]) / std[...,None,None]
def denormalize(x, mean,std): return x*std[...,None,None] + mean[...,None,None]

def normalize_batch(b, mean, std, do_y=False):
    x,y = b
    x = normalize(x,mean,std)
    if do_y: y = normalize(y,mean,std)
    return x,y

def normalize_funcs(mean, std, do_y=False, device=None):
    if device is None: device=default_device
    return (partial(normalize_batch, mean=mean.to(device),std=std.to(device), do_y=do_y),
            partial(denormalize,     mean=mean,           std=std))



In [ ]:
#imagenet mean/std
default_mean, default_std = Tensor([0.485, 0.456, 0.406]), Tensor([0.229, 0.224, 0.225])
default_norm,default_denorm = normalize_funcs(default_mean,default_std)

bs = 64
data = DataBunch.create(train_tds, valid_tds, bs=bs, dl_tfms=default_norm)


In [ ]:
x,y = next(iter(data.train_dl))
x = x.cpu()
y = y.cpu()
x = default_denorm(x)
#y = default_denorm(y)
show_xy_images(x,y,6, figsize=(9,10))
x.shape, y.shape

In [ ]:
#export
from torchvision.models import resnet34

model_meta = {
    resnet34:[8,6]
}

f = resnet34
cut,lr_cut = model_meta[f]

def cut_model(m, cut):
    return list(m.children())[:cut] if cut else m

def get_base():
    layers = cut_model(f(True), cut)
    return nn.Sequential(*layers)

def dice(pred, targs):
    pred = (pred>0).float()
    return 2. * (pred*targs).sum() / (pred+targs).sum()

def accuracy(out, yb):
    preds = torch.max(out, dim=1)[1]
    return (preds==yb).float().mean()

USE_GPU = torch.cuda.is_available()
def to_gpu(x, *args, **kwargs):
    '''puts pytorch variable to gpu, if cuda is available and USE_GPU is set to true. '''
    return x.cuda(*args, **kwargs) if USE_GPU else x

class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()
        
class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, 2, stride=2)
        self.bn = nn.BatchNorm2d(n_out)
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p))
    
class Unet34(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.sfs = [SaveFeatures(rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(512,256,256)
        self.up2 = UnetBlock(256,128,256)
        self.up3 = UnetBlock(256,64,256)
        self.up4 = UnetBlock(256,64,256)
        self.up5 = UnetBlock(256,3,16)
        self.up6 = nn.ConvTranspose2d(16, 1, 1)
        
    def forward(self,x):
        inp = x
        x = F.relu(self.rn(x))
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x, inp)
        x = self.up6(x)
        return x #[:,0]
    
    def close(self):
        for sf in self.sfs: sf.remove()

            
class UnetModel():
    def __init__(self,model,name='unet'):
        self.model,self.name = model,name

    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model.rn), [lr_cut]))
        return lgs + [children(self.model)[1:]]
    
class UnetModel():
    def __init__(self,model,name='unet'):
        self.model,self.name = model,name

    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model.rn), [lr_cut]))
        return lgs + [children(self.model)[1:]]

In [ ]:
m_base = get_base()
model = to_gpu(Unet34(m_base))
learn = Learner(data, model)
learn.metrics = [dice]
learn.loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
sched = OneCycleScheduler(learn, 0.5, 10)
learn.fit(10, 0.1, callbacks=[sched])

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
x,y = next(iter(data.valid_dl))
py = learn.model(x)
py = py.detach()

In [ ]:
show_image(y[0]), show_image(py[0]>0)

In [ ]:
learn.save('unet_128')

In [ ]:
size=512
bs = 8

train_tds, valid_tds = transform_datasets(train_ds, valid_ds, tfms, tfm_y=True, y_class=ImageMask, size=size)
data = DataBunch.create(train_tds, valid_tds, bs=bs, dl_tfms=default_norm)
learn = Learner(data, model)
learn.metrics = [dice]
learn.loss_fn = nn.BCEWithLogitsLoss()
learn.load('unet_128')

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
sched = OneCycleScheduler(learn, 0.1, 10)
learn.fit(10, 0.03, callbacks=[sched])

In [ ]:
learn.save('unet_512')

In [ ]:
apply_tfms??